In [1]:
import sympy as sp
from numpy import array
import numpy as np
from sympy import init_printing
init_printing(use_latex='mathjax')
import scipy
from sklearn.metrics import mean_squared_error

sqrt2 = np.sqrt(2)
sqrt3 = np.sqrt(3)
upper_triangle_idx  =np.triu_indices(5)

In [2]:
# load data
hill_data_path = "../data_6x6/processed_data/transverse_iso_6x6_bingo.csv"
hill_transposed_data_path = "../data_6x6/processed_data/transverse_iso_6x6_bingo_transpose.csv"
data = np.loadtxt(hill_data_path)
transposed_data = np.loadtxt(hill_transposed_data_path)

In [3]:
# data, transposed_data = remove_hydrostatic_dependence(data_pre, transposed_data_pre)
eqps_0_data = np.isclose(data[:,6],0.0)
#print(eqps_0_data)
data = data[eqps_0_data,:]
eqps_0_trans_data = np.isclose(transposed_data[:,6],0.0)
transposed_data = transposed_data[eqps_0_trans_data, :]
data, transposed_data

(array([[  3.16227766, -31.64854634,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ],
        [  3.16227766,  36.85700366,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ],
        [  3.16227766,   0.        , -55.92972971,   0.        ,
           0.        ,   0.        ,   0.        ],
        [  3.16227766,   0.        ,  59.09200737,   0.        ,
           0.        ,   0.        ,   0.        ],
        [  3.16227766,   0.        ,   0.        , -49.78704651,
           0.        ,   0.        ,   0.        ],
        [  3.16227766,   0.        ,   0.        ,  49.78704651,
           0.        ,   0.        ,   0.        ],
        [  3.16227766,   0.        ,   0.        ,   0.        ,
         -49.78704651,   0.        ,   0.        ],
        [  3.16227766,   0.        ,   0.        ,   0.        ,
          49.78704651,   0.        ,   0.        ],
        [  3.16227766,   0.        ,   0.        ,   0.        ,

In [4]:
def objective(params):
    #print(params)
    # fill a P matrix that has the params:
    candidate_P = np.array( [[ params[0], params[1], params[2], 0, 0, 0 ],
                             [ params[1], params[3], params[4], 0, 0, 0 ],
                             [ params[2], params[4], params[5], 0, 0, 0 ],
                             [ 0, 0, 0, params[6], 0, 0],
                             [ 0, 0, 0, 0, params[7], 0],
                             [0, 0, 0, 0, 0, params[8] ]
                             ] )
    #print(candidate_P)
    residual = np.zeros(len(data))
    for i, stress_vec in enumerate(data):
        sv = stress_vec[0:6]
        current_ys =  sv @ candidate_P @ sv 
        error = current_ys - 1

        residual[i] = np.abs(error)
    #print(scipy.linalg.norm(residual))
    return np.sum(np.abs(residual))
    


In [7]:
initial_guess = np.array([ 8.5, -7, -1.5, 8.5, -1.5, 3, 1.0, 1.0, 1.0 ])
result = scipy.optimize.minimize(objective, initial_guess, method = 'Nelder-Mead', options={'fatol':1e-19, 'maxiter':1e5})
print(result)

       message: Maximum number of iterations has been exceeded.
       success: False
        status: 2
           fun: 46307.152021674556
             x: [-2.599e-05 -2.587e+01 -5.297e+00  9.406e-04 -1.566e+00
                  1.307e-01  4.037e-04  5.363e-01 -8.761e-04]
           nit: 100000
          nfev: 1072756
 final_simplex: (array([[-2.599e-05, -2.587e+01, ...,  5.363e-01,
                        -8.761e-04],
                       [-2.599e-05, -2.587e+01, ...,  5.363e-01,
                        -8.761e-04],
                       ...,
                       [-2.599e-05, -2.587e+01, ...,  5.363e-01,
                        -8.761e-04],
                       [-2.599e-05, -2.587e+01, ...,  5.363e-01,
                        -8.761e-04]]), array([ 4.631e+04,  4.631e+04,  4.631e+04,  4.631e+04,
                        4.631e+04,  4.631e+04,  4.631e+04,  4.631e+04,
                        4.631e+04,  4.631e+04]))


In [6]:
params = result.x
candidate_P = np.array( [[ params[0], params[1], params[2], 0, 0, 0 ],
                            [ params[1], params[3], params[4], 0, 0, 0 ],
                            [ params[2], params[4], params[5], 0, 0, 0 ],
                            [ 0, 0, 0, params[6], 0, 0],
                            [ 0, 0, 0, 0, params[7], 0],
                            [0, 0, 0, 0, 0, params[8] ]
                            ] )
print(candidate_P)
residual = np.zeros(len(data))
for i, stress_vec in enumerate(data):
    sv = stress_vec[0:6]
    current_ys =  sv @ candidate_P @ sv 
    error = current_ys - 1

    residual[i] = error
print(residual)

[[ 8.52233550e-04 -2.58825173e+01 -5.29522186e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-2.58825173e+01  8.50000001e-04 -1.56365609e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-5.29522186e+00 -1.56365609e+00  1.31029009e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.00000002e-04
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   5.37110530e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -6.22447527e-05]]
[ 5.18058176e+03 -6.03315923e+03  2.28196728e+03 -1.52244263e+03
  2.23399897e-05  2.23399897e-05  1.33037125e+03  1.33037125e+03
 -1.30005603e+00 -1.30005603e+00  5.18058398e+03 -6.03315622e+03
  9.61998002e+02 -1.27842084e+02  4.50458582e-09  4.50458582e-09
  1.33037123e+03  1.33037123e+03 -1.30007836e+00 -1.30007836e+00
  1.12971166e+03 -1.16446881e+03  3.34496940e+02 -3.42918874e+02
  1